In [45]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

from tqdm.notebook import trange, tqdm

In [46]:
# Define the hyperparameters
learning_rate = 3e-4  # Learning rate for the optimizer
nepochs = 120  # Number of training epochs
batch_size = 64  # Batch size for training

output_size = 2

max_len = 64  # Maximum length of input sequences
chess_piece_type_count = 13 # six for each player and one for blank

In [47]:
board_data = torch.load("boards3-1.pt")
label_data = torch.load("labels3-1.pt")

print("board length: ", len(board_data))
print("label length: ", len(label_data))

board length:  658385
label length:  658385


In [48]:
class CustomChessDataset(Dataset):
    def __init__(self):
        pass

    def __len__(self):
        return len(board_data)

    def __getitem__(self, idx):
        label = label_data[idx]
        text = board_data[idx]
        return torch.tensor([label, 1.0 - label], dtype=torch.float), text

In [49]:
import torch.utils.data as data

full_dataset = CustomChessDataset()

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

print("Train size: ", train_size)
print("Test size: ", test_size)

train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Data Loaders:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Train size:  526708
Test size:  131677


## Create LSTM Model

In [50]:
from chess_model import LSTM

## Initialise Model and Optimizer

In [51]:
device = 'cuda'

# Define the size of the hidden layer and number of LSTM layers
hidden_size = 64
num_layers = 3

# Create the LSTM classifier model
lstm_classifier = LSTM(num_emb=chess_piece_type_count, output_size=2, num_layers=num_layers, hidden_size=hidden_size).to(device)

#lstm_classifier = ChessLSTM(chess_piece_type_count, hidden_size, num_layers, output_size, num_emb=chess_piece_type_count).to(device)

# Initialize the optimizer with Adam optimizer
optimizer = optim.Adam(lstm_classifier.parameters(), lr=learning_rate)

# Define the loss function as CrossEntropyLoss for classification
loss_fn = nn.BCEWithLogitsLoss()

# Initialize lists to store training and test loss, as well as accuracy
training_loss_logger = []
test_loss_logger = []
training_acc_logger = []
test_acc_logger = []

In [52]:
# Let's see how many Parameters our Model has!
num_model_params = 0
for param in lstm_classifier.parameters():
    num_model_params += param.flatten().shape[0]

print("-This Model Has %d (Approximately %d Million) Parameters!" % (num_model_params, num_model_params//1e6))

-This Model Has 100802 (Approximately 0 Million) Parameters!


## Training

In [53]:
for epoch in range(nepochs):
    epoch_loss = 0
    epoch_accuracy = 0
    # Set model to training mode
    lstm_classifier.train()
    steps = 0
    
    # Iterate through training data loader
    for label, text in tqdm(train_loader):
        label = label.to(device)
        text = text.to(device)
        
        output = lstm_classifier(text)
        loss = loss_fn(output, label)
            
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accuracy Calculation
        probs = F.sigmoid(output)  # Convert to probabilities
        predicted_class = (probs > 0.5).float()  # Apply threshold (adjust as needed)
        target_class = (label > 0.5).float()  # Extract the "stronger" class from soft label
        acc = (predicted_class == target_class).float().mean()
        
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)
    
    # Set model to evaluation mode
    lstm_classifier.eval()
    
    # Iterate through test data loader
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for label, text in tqdm(valid_loader):
            label = label.to(device)
            text = text.to(device)

            val_output = lstm_classifier(text)
            val_loss = loss_fn(val_output, label)
            
            # Accuracy Calculation
            probs = F.sigmoid(val_output)  # Convert to probabilities
            predicted_class = (probs > 0.5).float()  # Apply threshold (adjust as needed)
            target_class = (label > 0.5).float()  # Extract the "stronger" class from soft label
            acc = (predicted_class == target_class).float().mean()
            
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)
            
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
            

  0%|          | 0/8230 [00:00<?, ?it/s]

  0%|          | 0/2058 [00:00<?, ?it/s]

Epoch : 1 - loss : 0.6553 - acc: 0.6120 - val_loss : 0.6462 - val_acc: 0.6315


  0%|          | 0/8230 [00:00<?, ?it/s]

  0%|          | 0/2058 [00:00<?, ?it/s]

...

  0%|          | 0/2058 [00:00<?, ?it/s]

Epoch : 119 - loss : 0.6112 - acc: 0.6972 - val_loss : 0.6351 - val_acc: 0.6659


  0%|          | 0/8230 [00:00<?, ?it/s]

  0%|          | 0/2058 [00:00<?, ?it/s]

Epoch : 120 - loss : 0.6113 - acc: 0.6971 - val_loss : 0.6352 - val_acc: 0.6672


In [54]:
torch.save(lstm_classifier, 'model3-1.pt')

In [55]:
import chess.pgn
from create_dataset import chess_board_to_text
import io


model = torch.load("model3-1.pt")

pgn = open("Nakamura.pgn")

game = chess.pgn.read_game(pgn)
header = chess.pgn.read_headers(pgn)
print(header)
while not game.is_end():
#game = game.end()
    game = game.next()
    tensor = chess_board_to_text(str(game.board()))
    tensor.unsqueeze_(0)
    with torch.no_grad():
        predictions = model(tensor.to(device))
        probabilities = F.sigmoid(predictions) 
        np_arr = probabilities.detach().cpu().numpy()
        print(predictions, np_arr)

Headers(Event='Wch U10', Site='Cannes', Date='1997.??.??', Round='2', White='Nakamura, Hikaru', Black='El Mikati, Mohamad', Result='1-0', WhiteElo='', BlackElo='', ECO='C11')
tensor([[-0.0146,  0.0146]], device='cuda:0') [[0.4963506 0.5036495]]
tensor([[-0.0114,  0.0114]], device='cuda:0') [[0.49714217 0.5028578 ]]
tensor([[-0.0166,  0.0166]], device='cuda:0') [[0.49585295 0.50414705]]
tensor([[-0.0271,  0.0271]], device='cuda:0') [[0.49323434 0.5067656 ]]
tensor([[-0.0763,  0.0763]], device='cuda:0') [[0.48092866 0.51907134]]
tensor([[-0.0484,  0.0484]], device='cuda:0') [[0.48790276 0.5120972 ]]
tensor([[-0.0428,  0.0428]], device='cuda:0') [[0.4892895 0.5107105]]
tensor([[-0.0374,  0.0374]], device='cuda:0') [[0.49065402 0.509346  ]]
tensor([[-0.0267,  0.0267]], device='cuda:0') [[0.49333423 0.5066657 ]]
tensor([[-0.0072,  0.0072]], device='cuda:0') [[0.4982078  0.50179213]]
tensor([[-0.0052,  0.0052]], device='cuda:0') [[0.4986938  0.50130624]]
tensor([[-0.0014,  0.0014]], device='